# Seyed Mohammad Amin Atyabi - 830402014 - HW 7

در این مسئله قصد داریم تا با استفاده از روش بیز ساده، به دسته بندی مجموعه داده ای که در اختیارمان قرار گرفته است اقدام کنیم.

در روش بیز ساده فرض ساده ساز مستقل بودن ویژگی ها را در نظر میگیرم و سپس با استفاده از این فرض میتوان به سادگی احتمال تعلق یک نمونه به یک کلاس را بدست آورد.

برای این تمرین لازم است تا مجموعه داده را به 5 بخش که در هر کدام 65 عدد نمونه است تقسیم کنیم.

برای آموزش، ابتدا دو مجموعه که مشخص کننده دموکرات یا آزادی خواه بودن نمونه است تشکیل میدهیم.

سپس در هر مجموعه با توجه به موضوع مورد بحث، احتمال آری یا خیر بودن رای با توجه به حزب سیاسی تعیین میشود.

در همین حین وزن و تمایل مورد بحث را نیز محاسبه کرده و ذخیره میکنیم.

In [1]:
import json
import pandas
from math import log
from scipy import io as sio


def split_data(data: pandas.DataFrame):
    splits = []
    for i in range(5):
        splits.append(data.iloc[i:i + 65, :])
    return splits


def train_model(data: pandas.DataFrame):
    model = {}
    P = data.loc[data['DEMOCRAT/LIBERAL'] == 1]
    len_p = len(P) + 2
    N = data.loc[data['DEMOCRAT/LIBERAL'] == 0]
    len_n = len(N) + 2
    for column in data.columns:
        if column == 'DEMOCRAT/LIBERAL': continue
        p_p = P.loc[P[column] == 1]
        p_n = N.loc[N[column] == 0]
        model[column] = {
            '11': (len(p_p) + 1) / len_p,
            '01': 1 - ((len(p_p) + 1) / len_p),
            '10': (len(p_n) + 1) / len_n,
            '00': 1 - ((len(p_n) + 1) / len_n)
        }
        desire = log(model[column]['11'] / (1 - model[column]['11'])) - log(model[column]['10'] / (1 - model[column]['10']))
        model[column]['bias'] = 1 if desire > 0 else 0
        model[column]['weight'] = abs(desire)
    return model


def evaluate_model(model, data: pandas.DataFrame):
    error = 0
    for _, row in data.iterrows():
        predict_1 = 1
        predict_0 = 1
        for column in data.columns:
            if column == 'DEMOCRAT/LIBERAL': continue
            predict_1 *= model[column][f'{row[column]}1']
            predict_0 *= model[column][f'{row[column]}0']
        prediction = 1 if predict_1 > predict_0 else 0
        if prediction != row['DEMOCRAT/LIBERAL']: error += 1
    return error


def print_model(model):
    print(json.dumps(model, indent=4))

در این بخش داده خود را وارد نرم‌افزار کرده و سرآیند داده گان را نیز تنظیم میکنیم.

In [2]:
columns = [
    'HANDICAPPED-INFANTS',
    'WATER-PROJECT-COST-SHARING',
    'ADOPTION-OF-THE-BUDGET-RESOLUTION',
    'PHYSICIAN-FEE-FREEZE',
    'EL-SALVADOR-AID',
    'RELIGIOUS-GROUPS-IN-SCHOOL',
    'ANTI-SATELLITE-TEST-BAN',
    'AID-TO-NICARAGUAN-CONTRAS',
    'MX-MISSILE',
    'IMMIGRATION',
    'SYNFUELS-CORP-CUTBACK',
    'EDUCATION-SPENDING',
    'SUPERFUND-RIGHT-TO-SUE',
    'CRIME',
    'DUTY-FREE-EXPORTS',
    'EXPORT-ADMINISTRATION-ACT-SOUTH-AFRICA',
    'DEMOCRAT/LIBERAL'
]

data = pandas.DataFrame(sio.loadmat('Data/vote.mat').get('Data'), columns=columns, dtype=int)

در این بخش ابتدا داده را به چند قسمت تقسیم میکنیم و سپس وارد به ازای هر بخش مدلی آموزش میدهیم و در برابر دیگر بخش ها به آزمون میگذاریم.

به ازای هر بخش میانگین تعداد خطا را نسبت به بخش های دیگر محاسبه کرد و بخشی که کمترین خطا را دارد ذخیره میکنیم.

In [3]:
splits = split_data(data)

best_model = None
best_model_error = 0

for i in range(len(splits)):
    train_data = None
    for j in range(len(splits)):
        if j == i: continue
        train_data = pandas.concat([train_data, splits[j]])
    model = train_model(train_data)
    error = evaluate_model(model, splits[i])
    if best_model is None or error < best_model_error:
        best_model = model
        best_model_error = error
    print(error)

34
34
34
34
34


در ادامه مدلی که کمترین خطا را داشت ملاحظه میکنید

In [4]:
print_model(best_model)

{
    "HANDICAPPED-INFANTS": {
        "11": 0.5666666666666667,
        "01": 0.43333333333333335,
        "10": 0.7631578947368421,
        "00": 0.23684210526315785,
        "bias": 0,
        "weight": 0.9018072660555754
    },
    "WATER-PROJECT-COST-SHARING": {
        "11": 0.5333333333333333,
        "01": 0.4666666666666667,
        "10": 0.43859649122807015,
        "00": 0.5614035087719298,
        "bias": 1,
        "weight": 0.3803914705560484
    },
    "ADOPTION-OF-THE-BUDGET-RESOLUTION": {
        "11": 0.86,
        "01": 0.14,
        "10": 0.8245614035087719,
        "00": 0.17543859649122806,
        "bias": 1,
        "weight": 0.26772745792223596
    },
    "PHYSICIAN-FEE-FREEZE": {
        "11": 0.08666666666666667,
        "01": 0.9133333333333333,
        "10": 0.043859649122807015,
        "00": 0.956140350877193,
        "bias": 1,
        "weight": 0.7268784014284555
    },
    "EL-SALVADOR-AID": {
        "11": 0.3,
        "01": 0.7,
        "10": 0.043859